![PPGI_UFRJ](https://github.com/zavaleta/Fundamentos_DS/blob/main/imagens/ppgi-ufrj.png?raw=1)
# Fundamentos de Ciência de Dados
## 2022 - BL2
### **Aluno:** Gilberto Gil, Saulo Andrade Almeida, Valquire

# Trabalho Final
---

In [13]:
#checking version machine architecture, OS, python and all libs used in this notebook
import platform as platform
import numpy as np
import pandas as pd
import os
import conda
import pydot
import prov

def checkingEnvironmentVersions(details=False):

    #definnig version of python and all libs used
    HOST_MACHINE_ARCHTECTURE_EXPECTED = 'x86_64'
    HOST_MACHINE_OS_EXPECTED = 'Linux'
    HOST_MACHINE_PLATFORM = 'Linux-5.15.0-47-generic-x86_64-with-glibc2.31'
    CONDA_VERSION_EXPECTED = '4.14.0'
    CONDA_DEFAULT_ENV_EXPECTED = 'drecvmenv'
    PYTHON_VERSION_EXPECTED = '3.9.12'
    NUMPY_LIB_VERSION_EXPECTED = '1.21.5'
    PANDAS_LIB_VERSION_EXPECTED = '1.4.2'
    PYDOT_LIB_VERSION_EXPECTED = '1.4.2'
    PROV_LIB_VERSION_EXPECTED = '2.0.0'
    

    if details: 
        print('Host Machine Architecture:', platform.machine())
        print('Host Machine OS:', platform.system())
        print('Conda Version:', conda.__version__)
        print('Conda default env:', os.environ['CONDA_DEFAULT_ENV'])
        print('Python Version:', platform.python_version())
        print('NumPy Lib Version:', np.__version__)
        print('Pandas Lib Version:', pd.__version__)
        print('PyDot Lib Version:', pydot.__version__)
        print('Prov Lib Version:', prov.__version__)
        
    #checking versions
    try:
        #checking Machine Architecute expected
        assert platform.machine() == HOST_MACHINE_ARCHTECTURE_EXPECTED

        #checking OS expected
        assert platform.system() == HOST_MACHINE_OS_EXPECTED
        
        #checking conda version
        assert conda.__version__ == CONDA_VERSION_EXPECTED
        
        #checking conda default environment
        assert os.environ['CONDA_DEFAULT_ENV'] == CONDA_DEFAULT_ENV_EXPECTED    

        #checking python version
        assert platform.python_version() == PYTHON_VERSION_EXPECTED

        #checking numpy lib version
        assert np.__version__ == NUMPY_LIB_VERSION_EXPECTED  

        #checking Pandas lib version
        assert pd.__version__ == PANDAS_LIB_VERSION_EXPECTED
        
        #checking pydot version
        assert pydot.__version__ == PYDOT_LIB_VERSION_EXPECTED
        
        #checking prov version
        assert prov.__version__ == PROV_LIB_VERSION_EXPECTED
    except:
        #if any assert fail, or something else get wrong during verification
        if details: print('Something is wrong!')
        return False
    else:
        #if pass all asserts
        if details: print('All versions are correct!')
        return True

In [61]:
#Utilizado ambiente python 3.9
import pandas as pd
import numpy as np
#from datetime import datetime
#dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

def loadCsv(path, file):
  if path == 'data/cad-emp/':  # condição para fazer leitura do csv cadastro de companias
    return pd.read_csv(path+file, sep=';', header=0, encoding="ISO-8859-1")
  return pd.read_csv(path+file, sep=';', header=0, encoding="ISO-8859-1", parse_dates=['DT_REFER', 'DT_INI_EXERC', 'DT_FIM_EXERC'], 
                     converters={'CD_CONTA': str})

def loadCompanyInfo():
    return loadCsv("data/cad-emp/", "cad_cia_aberta.csv")

def allHistoricalYears():
    return [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

def loadDreItr(years):
    tempDF = pd.DataFrame() 
    for year in years:
        tempDF = pd.concat([tempDF, loadCsv("data/dre-itr/","itr_cia_aberta_DRE_con_{0}.csv".format(year))], ignore_index=True)
    return tempDF

def loadDreDfp(years):
    tempDF = pd.DataFrame() 
    for year in years:
        tempDF = pd.concat([tempDF, loadCsv("data/dre-dfp/",f'dfp_cia_aberta_DRE_con_{year}.csv')], ignore_index=True)
    return tempDF

def loadAllDreItr():
    return loadDreItr(allHistoricalYears())

def loadAllDreDfp():
    return loadDreDfp(allHistoricalYears())


def executeExperiment():
    companyInfoDF = loadCompanyInfo()
    dreItrDF = loadAllDreItr()
    dreDfpDF = loadAllDreDfp()
    print("Executando o experimento")
    print(f"Carregando Datasets. CadEmp:{companyInfoDF.shape}, DreItr:{dreItrDF.shape} e DrePfp:{dreDfpDF.shape} ")
    return companyInfoDF, dreItrDF, dreDfpDF
    
    

In [62]:
import sys, subprocess, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image

def createProvenance(agent, entity, activity, graph):
    # Creating an empty provenance document
    docProv = ProvDocument()

    # Declaring namespaces for various prefixes used in the excution of Randon Walk Experiment
    docProv.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')
    docProv.add_namespace('prov', 'http://www.w3.org/ns/prov#')
    docProv.add_namespace('void', 'http://vocab.deri.ie/void#')
    docProv.add_namespace('cvm', 'https://www.gov.br/cvm/pt-br')
    docProv.add_namespace('cvm-cademp', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2011', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2012', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2013', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2014', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2015', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2016', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2017', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2018', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2019', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2020', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-itr-2021', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    docProv.add_namespace('cvm-dre-dfp-2010', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/')

    
    
    # Creating all entity
    entDsCadEmp = docProv.entity('cvm-cademp:cad_cia_aberta.csv', {'prov:label': 'Dataset com dados da empresas listadas na CVM', 'prov:type': 'void:Dataset'})
    
    entDreItr = docProv.entity('cvm:dre-trimestral', {'prov:label': 'Documrnto que representa o conceito de DREs do tipo Trimestral', 'prov:type': 'foaf:Document'})
    entDsDreItr2011 = docProv.entity("cvm-dre-itr-2011:itr_cia_aberta_2011.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2011', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2011:itr_cia_aberta_2011.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2011:itr_cia_aberta_DRE_con_2011.csv', 'cvm-dre-itr-2011:itr_cia_aberta_2011.zip')
    entDsDreItr2012 = docProv.entity("cvm-dre-itr-2012:itr_cia_aberta_2012.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2012', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2012:itr_cia_aberta_2012.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2012:itr_cia_aberta_DRE_con_2012.csv', 'cvm-dre-itr-2012:itr_cia_aberta_2012.zip')
    entDsDreItr2013 = docProv.entity("cvm-dre-itr-2013:itr_cia_aberta_2013.zip", {'prov:label': 'Dataset com DRE trimestrais, do ano de 2013', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-itr-2013:itr_cia_aberta_2013.zip', 'cvm:dre-trimestral')
    docProv.wasDerivedFrom('cvm-dre-itr-2013:itr_cia_aberta_DRE_con_2013.csv', 'cvm-dre-itr-2013:itr_cia_aberta_2013.zip')

    
    
    entDreDfp = docProv.entity('cvm:dre-anual', {'prov:label': 'Documento que representa o conceito de DREs do tipo Anual', 'prov:type': 'foaf:Document'})
    entDsDreDfp2010 = docProv.entity("cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip", {'prov:label': 'Dataset com DRE anual, do ano de 2010', 'prov:type': 'void:Dataset'})
    docProv.wasDerivedFrom('cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip', 'cvm:dre-anual')
    docProv.wasDerivedFrom('cvm-dre-dfp-2010:dfp_cia_aberta_DRE_con_2010.csv', 'cvm-dre-dfp-2010:dfp_cia_aberta_2010.zip')
    
    

    # Creating all Agents  
    agntCvm = docProv.agent("cvm:CVM", 
                            {"prov:type":"prov:Organization", "foaf:name":"Comissão de Valores Mobiliários"})
    
    # create activity of dataset creations
    actvCreateDs = docProv.activity("cvm:create-dataset")    
    
    # Generation
    docProv.wasGeneratedBy(entDsCadEmp, actvCreateDs)
    docProv.wasGeneratedBy(entDreItr, actvCreateDs)
    docProv.wasGeneratedBy(entDreDfp, actvCreateDs)
    
    docProv.wasAssociatedWith(actvCreateDs, agntCvm)

    ### END - Registering Retrospective Provenance 

    ### Optional outputs ####

    #Generating the outup - a  Provenance Graph
    dot = prov_to_dot(docProv)
    graph = graph+".png"
    dot.write_png(graph)

    #Generating the Serialization - Output XML
    docProv.serialize(entity + ".xml", format='xml') 

    #Generating the Serialization - Output Turtle
    docProv.serialize(entity + ".ttl", format='rdf', rdf_format='ttl')

In [63]:
agent    = "Grupo02-Gil-Saulo-Valquire"                                         #PROV-Agent
entity   = "DREs-CVM"                                                           #PROV-Entity
activity = "TrabalhoFinalDisciplinaFundamentoDS-2022-2"                         #PROV-Activity
graph = entity                                                                  #PROV-Graph

def main():
    if(checkingEnvironmentVersions(True)):
        executeExperiment()
        createProvenance(agent, entity, activity, graph)
        
# main() # condição suspensa para poder rodar no notebook local

# Image(graph+\".png\") # condição suspensa para poder rodar no notebook local

In [ ]:
cadastro, informe_itr, informe_dfp = executeExperiment()

In [ ]:
cadastro.info()
informe_itr.info()
informe_dfp.info()

In [ ]:
# filtrar as empresas com cadastro ativo
cadastro = cadastro[cadastro['SIT'] == 'ATIVO']

# deletar colunas para o merge
cadastro.drop(['DENOM_SOCIAL', 'DENOM_COMERC', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'MOTIVO_CANCEL', 'SIT', 
'DT_INI_SIT', 'TP_MERC', 'CATEG_REG', 'DT_INI_CATEG', 'SIT_EMISSOR', 'DT_INI_SIT_EMISSOR', 'CONTROLE_ACIONARIO', 
'TP_ENDER', 'LOGRADOURO', 'COMPL', 'BAIRRO', 'MUN', 'UF', 'PAIS', 'CEP', 'DDD_TEL', 'TEL', 'DDD_FAX', 'FAX', 
'EMAIL', 'TP_RESP', 'RESP', 'DT_INI_RESP', 'LOGRADOURO_RESP', 'COMPL_RESP', 'BAIRRO_RESP', 'MUN_RESP', 'UF_RESP', 
'PAIS_RESP', 'CEP_RESP', 'DDD_TEL_RESP', 'TEL_RESP', 'DDD_FAX_RESP', 'FAX_RESP', 'EMAIL_RESP', 'CNPJ_AUDITOR', 
'AUDITOR'], axis=1, inplace=True)

# deletar os registros duplicados, que possuem mesmos cnpj e códigos cvm
print('Antes:', len(cadastro))
cadastro.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM'], keep = 'last', inplace=True)
print('Depois:', len(cadastro))

#cadastro.style

In [ ]:
# filtrar últimos informes trimestrais
informe_itr = informe_itr[informe_itr['ORDEM_EXERC'] == 'ÚLTIMO']

# fazer merge entre cadastro e informe itr, criando o dataset obinvest parcial
dataset_obinvest = pd.merge(informe_itr, cadastro, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)

# comparar registros antes e depois
print('Registros Antes:', len(informe_itr))
print('Registros Depois:', len(dataset_obinvest))
print('Diferença:', len(informe_itr) - len(dataset_obinvest))

dataset_obinvest

In [ ]:
# filtrar últimos informes anuais
informe_dfp = informe_dfp[informe_dfp['ORDEM_EXERC'] == 'ÚLTIMO']

# fazer merge entre cadastro e informe itr, criando o dataset obinvest
merge_dfp_cadastro = pd.merge(informe_dfp, cadastro, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)

# comparar registros antes e depois
print('Registros Antes:', len(informe_dfp))
print('Registros Depois:', len(merge_dfp_cadastro))
print('Diferença:', len(informe_dfp) - len(merge_dfp_cadastro))

merge_dfp_cadastro